<a href="https://colab.research.google.com/github/Sihan-A/PyTorch_Learning_Notebook/blob/main/GSDWP_04_First_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

`Runtime` -> `Change runtime type` -> `GPU`

# 2 First Neural Network with PyTorch